# Setup: Connectivity

In [1]:

from katana import remote
   #
from IPython.display import display
from ipywidgets import Layout, Textarea, VBox
   #
from libraries.SupportFor34 import f_init, f_enrich
    
    
my_client, my_graph = f_init()


print("My Client: %s" % (my_client))
print("My Graph:  %s" % (my_graph ))
   #
print()


My Client: <katana_enterprise.remote.sync_wrappers.Client object at 0x7f1642ab3460>
My Graph:  <_Graph my_graph, 7rYgkN2QekTsLbUmaw1FmNn32wyctwQHVqYup8ZNhQYv, 15>



#  Step 01: Enter a new Medical Description

In [2]:

#  Create a text entry area
#

l_layout   = Layout(flex = "0 1 auto", height = "100px", min_height = "100px", width = "auto")
l_text01   = Textarea(value="Odd mole on my ear", layout = l_layout, description = "Enter Text: ")
   #
l_vbox     = VBox([l_text01], layout = Layout(flex = "1 1 auto", width = "auto"))
display(l_text01)


Textarea(value='Odd mole on my ear', description='Enter Text: ', layout=Layout(flex='0 1 auto', height='100px'…

In [3]:

#  Enrich/decode text entry from above
#

l_return, l_pvid = f_enrich(l_text01.value.lower(), my_graph)

print("Unique Patient Vist Id: %s" % (l_pvid))
print("")
print("Medical Description encoding:")
print("")
display(l_return)


          0/? [?op/s]


Host 0 errors:
Traceback (most recent call last):
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py", line 111, in execute
    value = function(graph)
  File "/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py", line 512, in remote_import_function
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py", line 206, in _write_intermediate_parquet_files
    node_header = "\n".join(_write_dfs_to_parquet(node_df_infos, "nodes")) + "\n"
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py", line 194, in _write_dfs_to_parquet
    dask.dataframe.to_parquet(
  File "/opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/core.py", line 963, in to_parquet
    out = out.compute(**compute_kwargs)
  File "/opt/miniconda/lib/python3.8/site-packages/dask/bas

ValueError: Failed to convert partition to expected pyarrow schema:
    `ArrowInvalid("Could not convert 'Ear structure' with type str: tried to convert to double", 'Conversion failed for column preferred_term with type object')`

Expected partition schema:
    id: string
    entity_id: string
    LABEL: string
    preferred_term: double

Received partition schema:
    id: string
    entity_id: string
    LABEL: string
    preferred_term: string

This error *may* be resolved by passing in schema information for
the mismatched column(s) using the `schema` keyword in `to_parquet`.

# Step 02:  Examine the Graph

In [ ]:

#  Produce a visual graph

from katana_visualization_widget import GraphVisOptions, NodeVisOption, EdgeVisOption, ANY

l_options = GraphVisOptions(
   node_options = [
      NodeVisOption("PatientVisit",     label="sample_name"      ),
      NodeVisOption("UmlsEntity",       label="id"               ),
      NodeVisOption("UmlsVocabulary",   label="id"               ),
   ],
   #  edge_options = [
   #     EdgeVisOption(["VISIT_CONTAINS"], label="start_id"),
   #     EdgeVisOption(["ALSO_CODED_AS" ], label="start_id"),
   #  ]
   )
    
    
l_query  = """

   MATCH (pv: PatientVisit) - [vc: VISIT_CONTAINS] -> (ue: UmlsEntity) - [aca: ALSO_CODED_AS] -> (uv: UmlsVocabulary),
   WHERE
      (pv.id = '{0}')
   RETURN pv, vc, ue, ms, ios
   
   """.format(l_pvid)

l_result = my_graph.query(l_query, contextualize=True)
    
l_result.view(graph_vis_options = l_options)
